# AWS Lambda Container Deployment with Hydraa FaaS Manager

This notebook demonstrates container based lambda function deployment using the Hydraa FaaS Manager with ECR (Elastic Container Registry).

## Prerequisites
- AWS credentials configured
- Docker installed and running
- Hydraa FaaS Manager installed
- Lambda function code with Dockerfile prepared
- ECR permissions (create repositories, push images)

In [ ]:
from src.hydraa_faas.faas_manager.manager import FaasManager
from hydraa.services import ServiceManager
from hydraa import proxy, AWS, Task
import os
from dotenv import load_dotenv
import shutil

## Setup AWS Credentials

Make sure your AWS credentials are set as environment variables:

In [ ]:
# load environment variables from .env file
load_dotenv()

# verify aws credentials are set
required_vars = ['ACCESS_KEY_ID', 'ACCESS_KEY_SECRET', 'AWS_REGION']
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    raise ValueError(f"missing environment variables: {missing_vars}. please set them in your .env file.")

## Initialize FaaS Manager

Create the provider manager and FaaS manager:

In [ ]:
# initialize provider manager
provider_mgr = proxy([AWS])

# create faas manager with optimized settings
faas_mgr = FaasManager(
    provider_mgr, 
    asynchronous=True,  # enable parallel container builds
    auto_terminate=True
)

# start service manager
try:
    service_mgr = ServiceManager([faas_mgr])
    service_mgr.start_services()
except FileExistsError as e:
    sandbox_path = str(e).split("'")[1]
    shutil.rmtree(sandbox_path)
    service_mgr = ServiceManager([faas_mgr])
    service_mgr.start_services()

## Create Container-Based Lambda Function Tasks

Create container lambda function tasks that will be built from dockerfiles and pushed to ecr.

### Directory Structure Expected:
```
container_functions/
├── function_0/
│   ├── Dockerfile
│   ├── app.py
│   ├── ... # other source code files etc.
│   └── requirements.txt
├── function_1/
│   ├── Dockerfile
│   ├── app.py
│   ├── ...
│   └── requirements.txt
└── ...
```

In [ ]:
tasks = []
for i in range(2):
    task = Task()
    task.source_path = f'./container_functions/function_{i}'
    task.memory = 512
    task.timeout = 60
    task.provider = AWS
    task.create_ecr_repo = True
    task.handler_name = 'app.lambda_handler'
    task.env_var = ['FUNCTION_ID=0', 'ENV=production']
    tasks.append(task)

## Deploy Container Lambda Functions

Submit all container tasks for deployment. With the optimized manager, this will:
1. create ecr repositories
2. build docker images in parallel (max 3 concurrent)
3. push images to ecr
4. deploy lambda functions using container images

In [ ]:
# submit all container tasks for deployment
faas_mgr.submit(tasks)

In [ ]:
# wait for all container functions to deploy
results = []
deployed_functions = []

for i, task in enumerate(tasks):
    try:
        result = task.result(timeout=600)  # 10 minute timeout for container builds
        results.append(result)
        deployed_functions.append(task)
    except Exception as e:
        results.append(f"failed: {e}")

print("results",results,'\n')
print("deployed functions",deployed_functions)

## Create Task Using Existing ECR Image

Example of deploying a Lambda function using an existing ECR image:

In [ ]:
# create a task using existing ecr image
existing_image_task = Task()

# use existing ecr image (replace with your actual ecr uri)
account_id = "12345678"  # replace with your aws account id
region = os.getenv('AWS_REGION', 'us-east-1')
existing_image_task.image = f"ecr://{account_id}.dkr.ecr.{region}.amazonaws.com/my-existing-lambda:latest"
existing_image_task.memory = 1024
existing_image_task.timeout = 30
existing_image_task.provider = AWS
existing_image_task.create_ecr_repo = False  # don't create new repo

# environment variables
existing_image_task.env_var = [
    'FUNCTION_TYPE=existing_image',
    'ENV=production'
]

# deploy the existing image task
# faas_mgr.submit(existing_image_task)
# existing_result = existing_image_task.result(timeout=300)

## Invoke Container Lambda Functions

Test the deployed container functions:

In [ ]:
# invoke the deployed container functions
successful_invocations = 0
invocation_results = []

for i, task in enumerate(deployed_functions):
    try:
        response = faas_mgr.invoke(
            task.name, 
            {
                'message': f'Hello from container function {i}!', 
                'iteration': i,
                'container_test': True,
                'data': {
                    'input_array': [1, 2, 3, 4, 5],
                    'process_type': 'container_processing'
                }
            }
        )
        
        invocation_results.append({
            'function': task.name,
            'status': response['StatusCode'],
            'memory_mb': task.memory,
            'success': True
        })
        successful_invocations += 1
            
    except Exception as e:
        invocation_results.append({
            'function': task.name,
            'error': str(e),
            'success': False
        })

print(invocation_results)

## Check Container Function Status and ECR Resources

Get information about deployed container functions and created ECR repositories:

In [ ]:
# list all deployed functions
functions = faas_mgr.list_functions()
function_count = sum(len(func_list) for func_list in functions.values() if isinstance(func_list, list))

# check which functions are container-based
container_functions = []
for provider, func_list in functions.items():
    if isinstance(func_list, list):
        for func in func_list:
            if func.get('PackageType') == 'Image':
                container_functions.append(func['FunctionName'])

print(container_functions)

In [ ]:
# check provider status and resource usage
status = faas_mgr.get_provider_status()

# display summary
for provider, provider_status in status.items():
    active_status = "Active" if provider_status['active'] else "Inactive"
    function_count = provider_status['functions_count']
    run_id = provider_status['run_id']
    
# provider status retrieved successfully

## Display Container Deployment Results

Show the results of our container Lambda deployment and testing:

In [ ]:
# display container deployment summary
container_deployment_summary = {
    'total_container_tasks': len(tasks),
    'successful_container_deployments': len(deployed_functions),
    'failed_container_deployments': len(tasks) - len(deployed_functions),
    'successful_container_invocations': successful_invocations,
    'total_functions_in_account': function_count,
    'container_functions_deployed': len(container_functions),
    'ecr_repositories_created': len([t for t in deployed_functions if hasattr(t, 'create_ecr_repo') and t.create_ecr_repo])
}

# show key metrics
for key, value in container_deployment_summary.items():
    formatted_key = key.replace('_', ' ').title()
    
# display memory configurations used
memory_configs = [task.memory for task in deployed_functions]
unique_memory_configs = sorted(set(memory_configs))

print(container_deployment_summary)

# get performance metrics from the optimized manager
performance_metrics = faas_mgr.get_performance_metrics()
print(f"\nperformance metrics: {performance_metrics}")

# clean up resources
service_mgr.shutdown_services()

## Container Function Examples

### Example Dockerfile for Lambda Container:

```dockerfile
FROM public.ecr.aws/lambda/python:3.9

# Copy requirements and install dependencies
COPY requirements.txt ${LAMBDA_TASK_ROOT}
RUN pip install -r requirements.txt

# Copy function code
COPY app.py ${LAMBDA_TASK_ROOT}

# Set the CMD to your handler
CMD ["app.lambda_handler"]
```

### Example Python Lambda Handler (app.py):

```python
import json
import os

def lambda_handler(event, context):
    function_id = os.environ.get('FUNCTION_ID', 'unknown')
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'message': f'Hello from container function {function_id}!',
            'event': event,
            'container_runtime': True,
            'environment': os.environ.get('ENV', 'development')
        })
    }
```

## Summary

This notebook demonstrated:

1**ECR Integration** - automatic ecr repository creation and image management with proper cleanup
2**Build Process** - docker image building from source code with dockerfiles (now parallelized)
3**Container Configuration** - memory allocation and environment variables for containers
4**Existing Image Deployment** - using pre-built ecr images
5**Container Function Testing** - invoking containerized lambda functions
6**Resource Management** - tracking ecr repositories and container resources with error recovery

### Some reasons on why the ecr deployemnt method is a little more flexible:

- **Larger Package Size**: up to 10gb vs 50mb for zip packages
- **Custom Runtime**: use any language or runtime environment
- **Complex Dependencies**: include system libraries and custom tools
- **Reusable Images**: share images across multiple functions
- **Parallel Builds**: faster deployment with concurrent container builds